In [2]:
import json

In [3]:
def load_data(filename):
    with open(filename) as input_file:
        return json.load(input_file)

In [4]:
load_data('data/line-following-controller-data-17-07-2021-20.31.41.json')

FileNotFoundError: [Errno 2] No such file or directory: 'line-following-controller-data-17-07-2021-20.31.41.json'